In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten, Conv2D, MaxPool2D, Dropout
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.preprocessing import image
from PIL import Image
from better_bing_image_downloader import downloader
import os
import shutil

In [ ]:
queries = ['Space', 'Applicances', 'Furniture', 'Vegetables', 'Walls', 'Screens', 'Plants', 'Fruits',
           'Boats', 'Nature', 'Buildings', 'Pets', 'Computers', 'Cars', 'Planes']

In [ ]:
for query in queries:
    downloader(query, limit=32, output_dir='/No_Faces_Image_Download', filter='photo', name=query) 

In [ ]:
def copy_files(src_dir, dst_dir, num_files):
    for root, dirs, files in os.walk(src_dir):
        for dir_name in dirs:
            dir_path = os.path.join(root, dir_name)
            files_in_dir = os.listdir(dir_path)
            selected_files = files_in_dir[:num_files]
    
            for file_name in selected_files:
                src_file = os.path.join(dir_path, file_name)
                dst_file = os.path.join(dst_dir, file_name)
                shutil.copy(src_file, dst_file)

In [5]:
src_dir = '/No_Faces_Image_Download'
dst_dir = '/FaceDetection/Train/No_Faces'
num_files = 26
copy_files(src_dir, dst_dir, num_files)

In [ ]:
src_dir = '/No_Faces_Image_Download'
dst_dir = '/FaceDetection/Test/No_Faces'
num_files = 6
copy_files(sir_dir, dst_dir, num_files)

In [ ]:
src_dir = '/Kaggle_Faces/train'
dst_dir = 'FaceDetection/Train/Faces'
num_files = 26
copy_files(sir_dir, dst_dir, num_files)

In [ ]:
src_dir = '/Kaggle_Faces/test'
dst_dir = 'FaceDetection/Test/Faces'
num_files = 6
copy_files(sir_dir, dst_dir, num_files)

In [ ]:
train_path = '/FaceDetection/Train'
valid_path = '/FaceDetection/Test'

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input, rescale=1./255, rotation_range=20, zoom_range=(0.8, 1.2), horizontal_flip=True).flow_from_directory(directory=train_path, target_size=(224, 224), classes=['Faces', 'No_Faces'], batch_size=64, shuffle=True)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input, rescale=1./255, rotation_range=20, zoom_range=(0.8, 1.2), horizontal_flip=True).flow_from_directory(directory=valid_path, target_size=(224, 224), classes=['Faces', 'No_Faces'], batch_size=64, shuffle=True)

In [ ]:
imgs, labels = next(train_batches)

In [ ]:
def PlotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20, 20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
PlotImages(imgs)
print(labels)

In [ ]:
base_model = VGG16(weights = 'imagenet', include_top=False, input_shape = (224, 224, 3))

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model = Sequential([
    base_model,
    Dropout(0.5),
    Dense(512),
    Dropout(0.5),
    Dense(256),
    Flatten(),
    Dense(units=2, activation='softmax'),
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='BinaryCrossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x=train_batches, validation_data=valid_batches, class_weight={0: 1.0, 1: 1.0}, epochs=5, verbose=1)

In [ ]:
def preprocess_image(img_path, target_size=(224, 224)):
    img = Image.open(img_path)
    img = img.resize(target_size)
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.vgg16.preprocess_input(img_array)
    return img_array

In [ ]:
def predict_image_class(img_path, model):
    preprocessed_img = preprocess_image(img_path)
    PlotImages(preprocessed_img)
    predictions = model.predict(preprocessed_img)
    return predictions

In [ ]:
img_path = '.../image.jpg'
predicted_class = predict_image_class(img_path, model)
print(f'model is this confident the image shows a face: {predicted_class[0][0]} and this confident the image does not show a face: {predicted_class[0][1]}')

In [ ]:
#model.save('FaceDetection.h5')